In [1]:
# Openfermionpyscf : Hamiltonian construction
import openfermion as of
from openfermionpyscf import run_pyscf

# pennylane : quantum circuit & simulation
import pennylane as qml

# pennylane numpy :autograd
from pennylane import numpy as np

# default numpy
import numpy as npy
import itertools

# helper function
from helper import *

import pprint
import pickle

In [2]:
# molecule setting
basis = 'sto-3g'
multiplicity = 1
bond_length=1.6

geometry = [('Li', (0., 0., 0.)), ('H', (0., 0., bond_length))]
molecule = of.MolecularData(geometry,    basis,    multiplicity)
molecule = run_pyscf(molecule)
print(geometry)

[('Li', (0.0, 0.0, 0.0)), ('H', (0.0, 0.0, 1.6))]


In [3]:
# molecule properties
num_spatial_orbitals=molecule.n_orbitals
num_qubits=molecule.n_qubits
num_electrons=molecule.n_electrons

n_orbitals=num_qubits
n_electrons=num_electrons
print(" #Spatial orbitals (Full Problem) :",num_spatial_orbitals)
print(" #Qubits (Full Problem) :",num_qubits)
print(" #Electrons (Full Problem) :",num_electrons)

 #Spatial orbitals (Full Problem) : 6
 #Qubits (Full Problem) : 12
 #Electrons (Full Problem) : 4


In [4]:
# Second quantized hamiltonian with orbitals reduction
r_orbitals=[0]
a_orbitals=[1,2,3,4,5]

hamiltonian = molecule.get_molecular_hamiltonian(occupied_indices=r_orbitals,active_indices=a_orbitals)

# Fermionic Hamiltonian <-> JW Qubit Hamiltonian
Ham = of.jordan_wigner(hamiltonian)

# Data type tranformation  (openfermion <-> pennylane)
Ham_in=qml.import_operator(Ham)

# eigenvalues /eigenvectors (mixed particles number)

eigenValues, eigenVectors = np.linalg.eig(npy.round(qml.matrix(Ham_in),11))
idx = eigenValues.argsort()[::1]   
eigenValues = eigenValues[idx].real
eigenVectors = eigenVectors[:,idx].real

In [5]:
n_orbitals=hamiltonian.n_qubits
n_electrons=num_electrons-len(r_orbitals)*2

print(" #Qubits (Reduction) :",n_orbitals)
print(" #Electrons (Reduction) :",n_electrons)

 #Qubits (Reduction) : 10
 #Electrons (Reduction) : 2


In [6]:
N_Op=qml.qchem.particle_number(n_orbitals)
N_Op=qml.matrix(N_Op).real
S_z_Op=qml.qchem.spinz(n_orbitals)
S_z_Op=qml.matrix(S_z_Op).real
S2_Op=qml.qchem.spin2(n_electrons,n_orbitals)
S2_Op=qml.matrix(S2_Op).real

In [7]:
ans_index=[]
ans_S=[]
ans_Sz=[]
for i in range(20):
    n_particles=round( npy.transpose(eigenVectors[:,i]) @ N_Op @ eigenVectors[:,i] )
    if n_particles==n_electrons:
        S2= npy.transpose(eigenVectors[:,i]) @ S2_Op @ eigenVectors[:,i]
        print('index :',i)
        print("Energy : ",eigenValues[i])
        print('S^2: ', np.round(S2))
        print("S: ", round(-0.5+ npy.sqrt(1/4+S2)))
        ans_S.append(round(-0.5+ npy.sqrt(1/4+S2)))
        print("Sz : ", npy.round((npy.transpose(eigenVectors[:,i]) @ S_z_Op @ eigenVectors[:,i]),4),'\n')
        ans_Sz.append(npy.round((npy.transpose(eigenVectors[:,i]) @ S_z_Op @ eigenVectors[:,i]),4))
        ans_index.append(i)
E_ans=[eigenValues[i] for i in ans_index]

index : 0
Energy :  -7.882096599925185
S^2:  0.0
S:  0
Sz :  -0.0 

index : 3
Energy :  -7.76600490847333
S^2:  2.0
S:  1
Sz :  -0.0 

index : 4
Energy :  -7.766004908472654
S^2:  2.0
S:  1
Sz :  -1.0 

index : 5
Energy :  -7.766004908472588
S^2:  2.0
S:  1
Sz :  1.0 

index : 6
Energy :  -7.748714845346588
S^2:  0.0
S:  0
Sz :  -0.0 

index : 11
Energy :  -7.716090531299758
S^2:  2.0
S:  1
Sz :  -0.0 

index : 12
Energy :  -7.716090531299721
S^2:  2.0
S:  1
Sz :  0.0 

index : 13
Energy :  -7.716090531297463
S^2:  2.0
S:  1
Sz :  0.9928 

index : 14
Energy :  -7.716090531297418
S^2:  2.0
S:  1
Sz :  0.8369 

index : 15
Energy :  -7.71609053129741
S^2:  2.0
S:  1
Sz :  -0.8806 

index : 16
Energy :  -7.7160905312974055
S^2:  2.0
S:  1
Sz :  -1.0 

index : 17
Energy :  -7.696586314304366
S^2:  0.0
S:  0
Sz :  -0.0 

index : 18
Energy :  -7.696586314304313
S^2:  0.0
S:  0
Sz :  0.0 



In [8]:
E_ans_sz=[]
for i in range(len(ans_index)):
    if ans_Sz[i]==0:
        E_ans_sz.append(np.round(eigenValues[ans_index[i]],8))

In [9]:
# All particle conseving SDs
all_SDs=generate_binary_strings_with_counts(n_orbitals,n_electrons)

# Calculate SDs energies (Also can get SDs energies from molecule-integrals)
N_qubits=n_orbitals
dev_n = qml.device("lightning.qubit", wires=N_qubits)
@qml.qnode(dev_n,diff_method=None)
def SDs_energy(vector):
    qml.BasisState(np.array(vector), wires=range(N_qubits))
    return qml.expval(Ham_in)

In [10]:
All_SD_sets=dict()
for i in all_SDs:
    print("M: ",i)
    temp=[]
    for j in all_SDs[i]:
        SD_in=binary_string_to_list(j)
        print(' SD: ', SD_in,'Index: ',binatodeci(SD_in),'Energy of SD: ',SDs_energy(SD_in))
        temp.append([SD_in,binatodeci(SD_in),np.round(npy.array(SDs_energy(SD_in)),8)])
    All_SD_sets[i]=temp
    print('\n')

M:  0
 SD:  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0] Index:  768 Energy of SD:  -7.861864769808651
 SD:  [1, 0, 0, 1, 0, 0, 0, 0, 0, 0] Index:  576 Energy of SD:  -7.708923516888847
 SD:  [1, 0, 0, 0, 0, 1, 0, 0, 0, 0] Index:  528 Energy of SD:  -7.659401490236341
 SD:  [1, 0, 0, 0, 0, 0, 0, 1, 0, 0] Index:  516 Energy of SD:  -7.659401490236339
 SD:  [1, 0, 0, 0, 0, 0, 0, 0, 0, 1] Index:  513 Energy of SD:  -7.358803586209315
 SD:  [0, 1, 1, 0, 0, 0, 0, 0, 0, 0] Index:  384 Energy of SD:  -7.708923516888847
 SD:  [0, 1, 0, 0, 1, 0, 0, 0, 0, 0] Index:  288 Energy of SD:  -7.659401490236341
 SD:  [0, 1, 0, 0, 0, 0, 1, 0, 0, 0] Index:  264 Energy of SD:  -7.659401490236342
 SD:  [0, 1, 0, 0, 0, 0, 0, 0, 1, 0] Index:  258 Energy of SD:  -7.358803586209315
 SD:  [0, 0, 1, 1, 0, 0, 0, 0, 0, 0] Index:  192 Energy of SD:  -7.178009115340791
 SD:  [0, 0, 1, 0, 0, 1, 0, 0, 0, 0] Index:  144 Energy of SD:  -7.230939462951836
 SD:  [0, 0, 1, 0, 0, 0, 0, 1, 0, 0] Index:  132 Energy of SD:  -7.23093946295183

In [11]:
# number of eigen-energies you are interested in
k_number=4

# sort
inputstates=[x[0] for x in sorted(All_SD_sets[0],reverse=False,key=lambda x:(x[2],-x[1]))][0:k_number]
pprint.pprint(sorted(All_SD_sets[0],reverse=False,key=lambda x:(x[2],-x[1])))
print('\n','Reference states : \n',inputstates)

[[[1, 1, 0, 0, 0, 0, 0, 0, 0, 0], 768, -7.86186477],
 [[1, 0, 0, 1, 0, 0, 0, 0, 0, 0], 576, -7.70892352],
 [[0, 1, 1, 0, 0, 0, 0, 0, 0, 0], 384, -7.70892352],
 [[1, 0, 0, 0, 0, 1, 0, 0, 0, 0], 528, -7.65940149],
 [[1, 0, 0, 0, 0, 0, 0, 1, 0, 0], 516, -7.65940149],
 [[0, 1, 0, 0, 1, 0, 0, 0, 0, 0], 288, -7.65940149],
 [[0, 1, 0, 0, 0, 0, 1, 0, 0, 0], 264, -7.65940149],
 [[1, 0, 0, 0, 0, 0, 0, 0, 0, 1], 513, -7.35880359],
 [[0, 1, 0, 0, 0, 0, 0, 0, 1, 0], 258, -7.35880359],
 [[0, 0, 1, 0, 0, 1, 0, 0, 0, 0], 144, -7.23093946],
 [[0, 0, 1, 0, 0, 0, 0, 1, 0, 0], 132, -7.23093946],
 [[0, 0, 0, 1, 1, 0, 0, 0, 0, 0], 96, -7.23093946],
 [[0, 0, 0, 1, 0, 0, 1, 0, 0, 0], 72, -7.23093946],
 [[0, 0, 0, 0, 1, 0, 0, 1, 0, 0], 36, -7.230763],
 [[0, 0, 0, 0, 0, 1, 1, 0, 0, 0], 24, -7.230763],
 [[0, 0, 0, 0, 1, 1, 0, 0, 0, 0], 48, -7.19702472],
 [[0, 0, 0, 0, 0, 0, 1, 1, 0, 0], 12, -7.19702472],
 [[0, 0, 1, 1, 0, 0, 0, 0, 0, 0], 192, -7.17800912],
 [[0, 0, 1, 0, 0, 0, 0, 0, 0, 1], 129, -7.15457019],
 [[

### Excitation Pools 

In [12]:
active_orbitals=[x for x in range(n_orbitals)]

sz = npy.array( [0.5 if (i % 2 == 0) else -0.5 for i in range(len(active_orbitals))])

Allsingles=list(itertools.permutations(active_orbitals,2))
Alldoubles=list(itertools.permutations(active_orbitals,4))

##### $\Delta$ Sz=0 excitations

In [13]:
singles=[]
for r,p in Allsingles:
    # delta sz=0 generalized excitations
    if sz[r]-sz[p]==0:
        singles.append([r,p])
        print((r,p))

(0, 2)
(0, 4)
(0, 6)
(0, 8)
(1, 3)
(1, 5)
(1, 7)
(1, 9)
(2, 0)
(2, 4)
(2, 6)
(2, 8)
(3, 1)
(3, 5)
(3, 7)
(3, 9)
(4, 0)
(4, 2)
(4, 6)
(4, 8)
(5, 1)
(5, 3)
(5, 7)
(5, 9)
(6, 0)
(6, 2)
(6, 4)
(6, 8)
(7, 1)
(7, 3)
(7, 5)
(7, 9)
(8, 0)
(8, 2)
(8, 4)
(8, 6)
(9, 1)
(9, 3)
(9, 5)
(9, 7)


In [14]:
doubles = []
for p, q, r, s in Alldoubles:
    if sz[p] + sz[q] - sz[r] - sz[s] == 0:
        doubles.append([p,q,r,s])
        print((p,q,r,s))

(0, 1, 2, 3)
(0, 1, 2, 5)
(0, 1, 2, 7)
(0, 1, 2, 9)
(0, 1, 3, 2)
(0, 1, 3, 4)
(0, 1, 3, 6)
(0, 1, 3, 8)
(0, 1, 4, 3)
(0, 1, 4, 5)
(0, 1, 4, 7)
(0, 1, 4, 9)
(0, 1, 5, 2)
(0, 1, 5, 4)
(0, 1, 5, 6)
(0, 1, 5, 8)
(0, 1, 6, 3)
(0, 1, 6, 5)
(0, 1, 6, 7)
(0, 1, 6, 9)
(0, 1, 7, 2)
(0, 1, 7, 4)
(0, 1, 7, 6)
(0, 1, 7, 8)
(0, 1, 8, 3)
(0, 1, 8, 5)
(0, 1, 8, 7)
(0, 1, 8, 9)
(0, 1, 9, 2)
(0, 1, 9, 4)
(0, 1, 9, 6)
(0, 1, 9, 8)
(0, 2, 4, 6)
(0, 2, 4, 8)
(0, 2, 6, 4)
(0, 2, 6, 8)
(0, 2, 8, 4)
(0, 2, 8, 6)
(0, 3, 1, 2)
(0, 3, 1, 4)
(0, 3, 1, 6)
(0, 3, 1, 8)
(0, 3, 2, 1)
(0, 3, 2, 5)
(0, 3, 2, 7)
(0, 3, 2, 9)
(0, 3, 4, 1)
(0, 3, 4, 5)
(0, 3, 4, 7)
(0, 3, 4, 9)
(0, 3, 5, 2)
(0, 3, 5, 4)
(0, 3, 5, 6)
(0, 3, 5, 8)
(0, 3, 6, 1)
(0, 3, 6, 5)
(0, 3, 6, 7)
(0, 3, 6, 9)
(0, 3, 7, 2)
(0, 3, 7, 4)
(0, 3, 7, 6)
(0, 3, 7, 8)
(0, 3, 8, 1)
(0, 3, 8, 5)
(0, 3, 8, 7)
(0, 3, 8, 9)
(0, 3, 9, 2)
(0, 3, 9, 4)
(0, 3, 9, 6)
(0, 3, 9, 8)
(0, 4, 2, 6)
(0, 4, 2, 8)
(0, 4, 6, 2)
(0, 4, 6, 8)
(0, 4, 8, 2)
(0, 4, 8, 6)
(0, 5, 1, 2)

In [15]:
T1=[]
for p,q in singles:
    generator1=of.FermionOperator(((p, 1), (q, 0)))
    generator1+=of.hermitian_conjugated(generator1)*-1
    T1.append(generator1)

In [16]:
T2=[]
for p,q,r,s in doubles:
    generator2 =of.FermionOperator(((p, 1), (q, 1), (r, 0), (s, 0)))
    generator2+=of.hermitian_conjugated(generator2)*-1
    T2.append(generator2)

In [17]:
QT1=[]
for Op in T1:
    if (of.jordan_wigner(Op) not in QT1) and (of.jordan_wigner(Op)*-1 not in QT1):
        QT1.append(of.jordan_wigner(Op))
QT11=[]
for Op in T1:
        QT11.append(of.jordan_wigner(Op))         

In [18]:
QT2=[]
for Op2 in T2:
    if (of.jordan_wigner(Op2) not in QT2) and (of.jordan_wigner(Op2)*-1 not in QT2):
        QT2.append(of.jordan_wigner(Op2))
QT22=[]
for Op2 in T2:
        QT22.append(of.jordan_wigner(Op2))          

In [19]:
aq_in=[[0,0],[0,1],[1,0],[1,1]]
in_states=[]
for i in range(len(inputstates)):
    temp=inputstates[i]+aq_in[i]
    in_states.append(temp)

In [20]:
a_qubits=int(np.ceil(np.log2(k_number)))
w=[0]*2**(n_orbitals+a_qubits)
weight_in=[4,3,2,1]
for i in range(len(in_states)):
    w[binatodeci(in_states[i])]=weight_in[i]

v_in=npy.array(w)/npy.linalg.norm(np.array(w))   
N_qubits=n_orbitals

# auxiliary qubits
dev2 = qml.device("lightning.qubit", wires=n_orbitals+a_qubits)

input_array=np.array(v_in)

In [21]:
@qml.qnode(dev2,diff_method='adjoint')
def pvqe(params):
    
    # QSP
    qml.QubitStateVector(np.array(input_array,requires_grad=False), wires=range(N_qubits+a_qubits))

    
    # modified UCCGSD
    for i in range(len(QT11)):
        H1_in=QT11[i]*1j
        qml.CommutingEvolution(qml.import_operator(H1_in),time=params[i])
    for k in range(len(QT22)):
        H2_in=QT22[k]*1j
        qml.CommutingEvolution(qml.import_operator(H2_in),time=params[len(QT11)+k])

    return qml.expval(Ham_in)

In [22]:
# Set up a optimizer, and run the pvqe
lr=0.02
b1=0.9
b2=0.999
opt = qml.AdamOptimizer(stepsize=lr,beta1=b1,beta2=b2)
max_iterations = 2000
conv_tol = 1e-9

print("Adam setting: ",'Learning rate: ',lr,'beta1: ',b1,'beta2: ',b2, 'eps: ',1e-8,'\n')

Adam setting:  Learning rate:  0.02 beta1:  0.9 beta2:  0.999 eps:  1e-08 



In [23]:
E_energy_ext=v_in[binatodeci(in_states[0])]**2*E_ans_sz[0]+ \
v_in[binatodeci(in_states[1])]**2*E_ans_sz[1]+ \
v_in[binatodeci(in_states[2])]**2*E_ans_sz[2]+\
v_in[binatodeci(in_states[3])]**2*E_ans_sz[3]
print(E_energy_ext)

-7.8239513239999985


In [24]:
params=np.zeros(len(QT11)+len(QT22))

In [25]:
EnsembleE=[]
#params = np.random.random(shape)


params_set=[]
for n in range(max_iterations):
    params, prev_energy = opt.step_and_cost(pvqe, params)
    params_set.append(params)
    Energy=pvqe(params)
    EnsembleE.append(Energy)
    conv = np.abs(EnsembleE[-1] - prev_energy)

    if n % 10 == 0:
        print(f"Step = {n},  Ensemble Energy = {EnsembleE[-1]:.8f} Ha")
    if conv <= conv_tol:
        print(f"Step = {n},  Ensemble Energy = {EnsembleE[-1]:.8f} Ha")
        break

Step = 0,  Ensemble Energy = -7.76376926 Ha
Step = 10,  Ensemble Energy = -7.81476866 Ha
Step = 20,  Ensemble Energy = -7.81997408 Ha
Step = 30,  Ensemble Energy = -7.82283439 Ha
Step = 40,  Ensemble Energy = -7.82348815 Ha
Step = 50,  Ensemble Energy = -7.82377864 Ha
Step = 60,  Ensemble Energy = -7.82389845 Ha
Step = 70,  Ensemble Energy = -7.82392952 Ha
Step = 80,  Ensemble Energy = -7.82394258 Ha
Step = 90,  Ensemble Energy = -7.82394852 Ha
Step = 100,  Ensemble Energy = -7.82395032 Ha
Step = 110,  Ensemble Energy = -7.82395098 Ha
Step = 120,  Ensemble Energy = -7.82395116 Ha
Step = 130,  Ensemble Energy = -7.82387695 Ha
Step = 140,  Ensemble Energy = -7.82380700 Ha
Step = 150,  Ensemble Energy = -7.82381760 Ha
Step = 160,  Ensemble Energy = -7.82394012 Ha
Step = 170,  Ensemble Energy = -7.82393861 Ha
Step = 180,  Ensemble Energy = -7.82394690 Ha
Step = 190,  Ensemble Energy = -7.82395041 Ha
Step = 200,  Ensemble Energy = -7.82395087 Ha
Step = 210,  Ensemble Energy = -7.82395128 Ha

In [26]:
E_energy_ext=v_in[binatodeci(in_states[0])]**2*E_ans_sz[0]+ \
v_in[binatodeci(in_states[1])]**2*E_ans_sz[1]+ \
v_in[binatodeci(in_states[2])]**2*E_ans_sz[2]+\
v_in[binatodeci(in_states[3])]**2*E_ans_sz[3]
print(E_energy_ext)

print("Exect ensemble: ", E_energy_ext)
print("Difference: ", pvqe(params_set[-1])-E_energy_ext)

-7.8239513239999985
Exect ensemble:  -7.8239513239999985
Difference:  2.0847847892468963e-09


In [27]:
dev3 = qml.device("lightning.qubit", wires=n_orbitals)

@qml.qnode(dev3,diff_method=None)
def pvqe_extract_state(ind_vector,params_in):
    
    # QSP (working qubit only)
    qml.BasisState(np.array(ind_vector), wires=range(N_qubits))

    # modified UCCGSD
    for i in range(len(QT11)):
        H1_in=QT11[i]*1j
        qml.CommutingEvolution(qml.import_operator(H1_in),time=params[i])
    for k in range(len(QT22)):
        H2_in=QT22[k]*1j
        qml.CommutingEvolution(qml.import_operator(H2_in),time=params[len(QT11)+k])
    
    
    qml.Barrier(wires=range(N_qubits))
    return qml.state()


dev3 = qml.device("lightning.qubit", wires=n_orbitals)

@qml.qnode(dev3,diff_method=None)
def pvqe_extract(ind_vector,params_in):
    
    # QSP (working qubit only)
    qml.BasisState(np.array(ind_vector), wires=range(N_qubits))

    # modified UCCGSD
    for i in range(len(QT11)):
        H1_in=QT11[i]*1j
        qml.CommutingEvolution(qml.import_operator(H1_in),time=params[i])
    for k in range(len(QT22)):
        H2_in=QT22[k]*1j
        qml.CommutingEvolution(qml.import_operator(H2_in),time=params[len(QT11)+k])
    
    
    qml.Barrier(wires=range(N_qubits))
    return qml.expval(Ham_in)


In [28]:
E_ans_sz

[-7.8820966,
 -7.76600491,
 -7.74871485,
 -7.71609053,
 -7.71609053,
 -7.69658631,
 -7.69658631]

In [29]:
for i in range(len(inputstates)):
    print('Reference state: ', inputstates[i])
    #print('Final state: \n' ,np.round(pvqe_extract_state(inputstates[i],params_set[-1]),8))
    id_energy= np.round(pvqe_extract(inputstates[i],params_set[-1]),8)
    print("\nPVQE Energy_{}: ".format(i), id_energy," \nDifference: ",id_energy-E_ans_sz[i])
    S2= npy.transpose(pvqe_extract_state(inputstates[i],params_set[-1]) @ S2_Op @ pvqe_extract_state(inputstates[i],params_set[-1]))
    #print('index :',i)
    print('S^2: ', np.round(S2))
    print("S: ", np.round(-0.5+ npy.sqrt(1/4+S2)))
    Sz=npy.transpose(pvqe_extract_state(inputstates[i],params_set[-1]) @ S_z_Op @ pvqe_extract_state(inputstates[i],params_set[-1]))
    print('Sz: ', np.round(Sz),'\n \n')
    print('-----------------------------------------------------------------------------')

Reference state:  [1, 1, 0, 0, 0, 0, 0, 0, 0, 0]

PVQE Energy_0:  -7.8820966  
Difference:  0.0
S^2:  0j
S:  0j
Sz:  0j 
 

-----------------------------------------------------------------------------
Reference state:  [1, 0, 0, 1, 0, 0, 0, 0, 0, 0]

PVQE Energy_1:  -7.76600491  
Difference:  0.0
S^2:  (2+0j)
S:  (1+0j)
Sz:  0j 
 

-----------------------------------------------------------------------------
Reference state:  [0, 1, 1, 0, 0, 0, 0, 0, 0, 0]

PVQE Energy_2:  -7.74871485  
Difference:  0.0
S^2:  0j
S:  0j
Sz:  (-0+0j) 
 

-----------------------------------------------------------------------------
Reference state:  [1, 0, 0, 0, 0, 1, 0, 0, 0, 0]

PVQE Energy_3:  -7.71609053  
Difference:  0.0
S^2:  (2+0j)
S:  (1+0j)
Sz:  0j 
 

-----------------------------------------------------------------------------


In [30]:
#save_file_name='LiH_{0}_{1}_params_3es.pickle'.format(sys_args,bond_length)
save_file_name='LiH_{0}_4es.pickle'.format(bond_length)
with open(save_file_name, 'wb') as f:
    pickle.dump(params_set, f)